In [1]:
import numpy
import pandas as pd
import json
from pprint import pprint
from splinter import Browser 
from bs4 import BeautifulSoup as bs
import time
import collections

In [2]:
data = pd.read_csv("NYCinspection.csv") 
data.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,50000598,UNCLE TED'S,MANHATTAN,163,BLEECKER STREET,10012.0,2127771395,Chinese,10/3/2016,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage-associated (...,Critical,46.0,NaN,NaN,2/6/2019,Cycle Inspection / Initial Inspection
1,50000786,DUNKIN' DONUTS,BROOKLYN,873,FLATBUSH AVENUE,11226.0,7189400080,Donuts,12/14/2017,Violations were cited in the following area(s).,10H,Proper sanitization not provided for utensil w...,Not Critical,10.0,NaN,NaN,2/6/2019,Cycle Inspection / Initial Inspection
2,41703747,M2N BUFFET,QUEENS,4012,82 STREET,11373.0,7188032488,American,1/27/2016,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,21.0,NaN,NaN,2/6/2019,Cycle Inspection / Initial Inspection
3,40394258,TOMMASO RESTAURANT,BROOKLYN,1464,86 STREET,11228.0,7182369883,Italian,10/24/2017,Violations were cited in the following area(s).,02G,Cold food item held above 41Âº F (smoked fish ...,Critical,27.0,B,10/24/2017,2/6/2019,Cycle Inspection / Re-inspection
4,50059515,STICKY'S FINGER JOINT,MANHATTAN,21,MAIDEN LN,10038.0,9177551770,Chicken,3/8/2018,Violations were cited in the following area(s).,06A,Personal cleanliness inadequate. Outer garment...,Critical,5.0,A,3/8/2018,2/6/2019,Cycle Inspection / Initial Inspection


In [3]:
data = data.drop(['CAMIS','BUILDING','PHONE','ZIPCODE','INSPECTION DATE','ACTION','INSPECTION TYPE'], axis=1)
data.head(5)


,DBA,BORO,STREET,CUISINE DESCRIPTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE
0,UNCLE TED'S,MANHATTAN,BLEECKER STREET,Chinese,04N,Filth flies or food/refuse/sewage-associated (...,Critical,46.0,NaN,NaN,2/6/2019
1,DUNKIN' DONUTS,BROOKLYN,FLATBUSH AVENUE,Donuts,10H,Proper sanitization not provided for utensil w...,Not Critical,10.0,NaN,NaN,2/6/2019
2,M2N BUFFET,QUEENS,82 STREET,American,10F,Non-food contact surface improperly constructe...,Not Critical,21.0,NaN,NaN,2/6/2019
3,TOMMASO RESTAURANT,BROOKLYN,86 STREET,Italian,02G,Cold food item held above 41Âº F (smoked fish ...,Critical,27.0,B,10/24/2017,2/6/2019
4,STICKY'S FINGER JOINT,MANHATTAN,MAIDEN LN,Chicken,06A,Personal cleanliness inadequate. Outer garment...,Critical,5.0,A,3/8/2018,2/6/2019


In [4]:
data = data.dropna()
data_gb = data.groupby(['DBA','STREET','BORO','CUISINE DESCRIPTION'])['VIOLATION CODE'].count()
data_gb.head()

DBA                           STREET             BORO       CUISINE DESCRIPTION                                             
#1 Chinese Restaurant         W 181ST ST         MANHATTAN  Chinese                                                              7
#1 GARDEN CHINESE RESTAURANT  PROSPECT PARK W    BROOKLYN   Chinese                                                              5
#1 Natural Juice Bar          SENECA AVE         QUEENS     Juice, Smoothies, Fruit Salads                                       8
#1 SABOR LATINO RESTAURANT    WHITE PLAINS ROAD  BRONX      Latin (Cuban, Dominican, Puerto Rican, South & Central American)    17
$ 1 SLICE PIZZA               JEROME AVE         BRONX      Pizza                                                                8
Name: VIOLATION CODE, dtype: int64

In [5]:
data_df = pd.DataFrame(data_gb)
data_df.reset_index
data_df.to_csv('nycrest.csv', sep=',', encoding='utf-8')

In [6]:
nyc_rest = pd.read_csv("nycrest.csv") 
nyc_rest.head(20)

,DBA,STREET,BORO,CUISINE DESCRIPTION,VIOLATION CODE
0,#1 Chinese Restaurant,W 181ST ST,MANHATTAN,Chinese,7
1,#1 GARDEN CHINESE RESTAURANT,PROSPECT PARK W,BROOKLYN,Chinese,5
2,#1 Natural Juice Bar,SENECA AVE,QUEENS,"Juice, Smoothies, Fruit Salads",8
3,#1 SABOR LATINO RESTAURANT,WHITE PLAINS ROAD,BRONX,"Latin (Cuban, Dominican, Puerto Rican, South &...",17
4,$ 1 SLICE PIZZA,JEROME AVE,BRONX,Pizza,8
5,$1.25 PIZZA,FULTON ST,BROOKLYN,Pizza,8
6,$5 TO $9 RESTAURANT,3RD AVE,BROOKLYN,Chinese,6
7,& PIZZA,BROADWAY,MANHATTAN,Pizza,2
8,& PIZZA,W 28TH ST,MANHATTAN,Pizza,4
9,'CESCA,W 75TH ST,MANHATTAN,Italian,3


In [7]:
nyc_rest = nyc_rest.rename(index=str, columns={"CUISINE DESCRIPTION": "TYPE", "VIOLATION CODE": "VIOLATIONS"})
nyc_rest.head()

,DBA,STREET,BORO,TYPE,VIOLATIONS
0,#1 Chinese Restaurant,W 181ST ST,MANHATTAN,Chinese,7
1,#1 GARDEN CHINESE RESTAURANT,PROSPECT PARK W,BROOKLYN,Chinese,5
2,#1 Natural Juice Bar,SENECA AVE,QUEENS,"Juice, Smoothies, Fruit Salads",8
3,#1 SABOR LATINO RESTAURANT,WHITE PLAINS ROAD,BRONX,"Latin (Cuban, Dominican, Puerto Rican, South &...",17
4,$ 1 SLICE PIZZA,JEROME AVE,BRONX,Pizza,8


In [8]:
nyc_rest = nyc_rest[nyc_rest.TYPE != 'CafÃ©/Coffee/Tea']
nyc_rest = nyc_rest[nyc_rest.TYPE != 'Juice, Smoothies, Fruit Salads']
nyc_rest = nyc_rest[nyc_rest.TYPE != 'Ice Cream, Gelato, Yogurt, Ices']
nyc_rest['Google_Results'] = " "
nyc_rest.head(20)

,DBA,STREET,BORO,TYPE,VIOLATIONS,Google_Results
0,#1 Chinese Restaurant,W 181ST ST,MANHATTAN,Chinese,7,
1,#1 GARDEN CHINESE RESTAURANT,PROSPECT PARK W,BROOKLYN,Chinese,5,
3,#1 SABOR LATINO RESTAURANT,WHITE PLAINS ROAD,BRONX,"Latin (Cuban, Dominican, Puerto Rican, South &...",17,
4,$ 1 SLICE PIZZA,JEROME AVE,BRONX,Pizza,8,
5,$1.25 PIZZA,FULTON ST,BROOKLYN,Pizza,8,
6,$5 TO $9 RESTAURANT,3RD AVE,BROOKLYN,Chinese,6,
7,& PIZZA,BROADWAY,MANHATTAN,Pizza,2,
8,& PIZZA,W 28TH ST,MANHATTAN,Pizza,4,
9,'CESCA,W 75TH ST,MANHATTAN,Italian,3,
10,'WICHCRAFT,BROADWAY,MANHATTAN,Sandwiches,5,


In [86]:
DBA = list(nyc_rest['DBA'])
ratings = list(nyc_rest['Google_Results'])
loc = list(nyc_rest["STREET"])
dba_ratings = dict(zip(DBA, ratings))
dba_loc = dict(zip(DBA,loc))
pprint(dba_loc)

{'#1 Chinese Restaurant': 'W 181ST ST',
 '#1 GARDEN CHINESE RESTAURANT': 'PROSPECT PARK W',
 '#1 SABOR LATINO RESTAURANT': 'WHITE PLAINS ROAD',
 '$ 1 SLICE PIZZA': 'JEROME AVE',
 '$1.25 PIZZA': 'FULTON ST',
 '$5 TO $9 RESTAURANT': '3RD AVE',
 '& PIZZA': 'W 28TH ST',
 "'CESCA": 'W 75TH ST',
 "'WICHCRAFT": 'BROADWAY',
 '(LEWIS DRUG STORE) LOCANDA VINI E OLII': 'GATES AVENUE',
 '(PUBLIC FARE) 81st street and central park west (Delacorte Theatre)': 'CENTRAL PARK WEST',
 '.TAO': '9TH AVE',
 '1 2 3 BURGER SHOT BEER': '10 AVENUE',
 '1 DARBAR': 'EAST   46 STREET',
 '1 EAST 66TH STREET KITCHEN': 'EAST   66 STREET',
 '1 HOTEL BROOKLYN BRIDGE': 'FURMAN ST',
 '1 HOTEL EMPLOYEE DINING KITCHEN #36': 'FURMAN ST',
 '1 OAK': 'WEST   17 STREET',
 '1 OR 8': 'SOUTH    2 STREET',
 '1 QUETZAL': 'NEW UTRECHT AVE',
 '1 STOP PATTY SHOP': 'AMSTERDAM AVENUE',
 '1 TASTE OF INDIA': 'LIBERTY AVE',
 '1.5 DAK GALBI INC.': 'NORTHERN BLVD',
 '10 02 Lounge and Restaurant': 'E TREMONT AVE',
 '10 CORSO COMO': 'FULTON ST',

 'AJO & OREGANO RESTAURANT': 'WHITE PLAINS RD',
 'AKAMI SUSHI': '1ST AVE',
 'AKDENIZ MEDITERRANEAN CUISINE': 'W 46TH ST',
 'AKI KING': 'E 23RD ST',
 'AKI SUSHI': 'YORK AVE',
 'AKI SUSHI AND GRILL': '2ND AVE',
 'AKIMOTO SUSHI': 'CHURCH STREET',
 'AKIN GUMP': 'BRYANT PARK',
 'AKINO': 'BROADWAY',
 'AKIYAMA JAPANESE RESTAURANT': '14 AVENUE',
 'AKO JAPANESE RESTAURANT': 'BEDFORD AVE',
 'AKROTIRI': '30 AVENUE',
 'AKSARAY TURKISH CAFE AND RESTAURANT': 'EAST   16 STREET',
 'AKURA SUSHI': 'E 31ST ST',
 'AKWAABA RESTAURANT': 'PARKSIDE AVENUE',
 'AL CHOLO WEST INDIAN BAKERY & HEALTH FOOD STORE': 'WHITE PLAINS RD',
 'AL DI LA': '5 AVENUE',
 'AL HIRSCHFELD THEATRE': 'WEST   45 STREET',
 'AL HORNO': 'PEARL ST',
 'AL HORNO LEAN MEXICAN': '2ND AVE',
 'AL HORNO LEAN MEXICAN KITCHEN': '1ST AVE',
 'AL HORNO LEAN MEXICAN KITCHEN INC': 'W 47TH ST',
 'AL NAR CAFE': 'UNION TPKE',
 'AL VAPORETTO': 'E 81ST ST',
 'AL VICOLETTO': 'E 17TH ST',
 "AL'S DELI": '7TH AVE',
 "AL'S FAMOUS PIZZERIA": 'ALLERTON AVE',
 'AL

 'ATRIUM CINEMAS': 'ARTHUR KILL ROAD',
 'ATRIUM DUMBO': 'MAIN STREET',
 'ATWOOD KITCHEN & BAR': '2ND AVE',
 'AU BON PAIN': 'WILLIAM ST',
 'AU BON PAIN (GRAB N GO)': 'W 168TH ST',
 'AU BON PAIN, ANDALE MEXICAN GRILL': 'UNIVERSITY AVENUE',
 'AU BON PAIN, CIH CAFE': 'OCEAN PARKWAY',
 'AU GOUT DU MAITRE RESTAURANT': 'FLATBUSH AVE',
 'AU JUS': '1ST AVE',
 "AU ZA'ATAR": 'AVENUE A',
 'AUBERGINE CAFFE': 'SKILLMAN AVENUE',
 'AUCTION HOUSE': 'EAST   89 STREET',
 'AUDI YANKEE CLUB': 'EAST  161 STREET',
 'AUDREY BAKERY & CAFE': 'CHATHAM SQ',
 'AUGIES DELI & PIZZERIA': 'CITY ISLAND AVE',
 'AUGURS WELL': 'ST MARKS PLACE',
 'AUGUST': 'LEXINGTON AVE',
 'AUGUST GATHERINGS': 'CANAL ST',
 'AUGUST WILSON THEATRE': 'WEST   52 STREET',
 'AUGUSTINE': 'BEEKMAN ST',
 "AUNT ANNIE'S": 'RICHMOND TERRACE',
 'AUNT BUTCHIES OF BROOKLYN': 'ARTHUR KILL RD',
 'AUNT BUTCHIES/ REVIVE BAR': 'JFK INTERNATIONAL AIRPORT',
 "AUNT GINNY'S": 'WOODWARD AVE',
 "AUNT JAKE'S": 'MULBERRY ST',
 'AUNT JAKES': 'WEST    8 STREET',
 "AUN

 'BEC': '8TH AVE',
 'BECCO': 'WEST   46 STREET',
 'BECCOFINO': 'MOSHOLU AVENUE',
 "BECKETT'S": 'PEARL STREET',
 'BECKYS BITES': 'E 7TH ST',
 'BECO': 'LORIMER STREET',
 'BED STUY': 'ATLANTIC AVENUE',
 'BED-STUY FISH FRY': 'UTICA AVE',
 'BED-STUY FISHFRY': 'NOSTRAND AVE',
 'BED-STUY PROVISIONS': 'GATES AVE',
 'BEDAWI CAFE': 'PROSPECT PARK WEST',
 'BEDFORD & CO': 'E 40TH ST',
 'BEDFORD CAFE & RESTAURANT': 'EAST BEDFORD PARK BOULEVARD',
 'BEDFORD FALLS': 'EAST   67 STREET',
 'BEDFORD HILL': 'FRANKLIN AVENUE',
 'BEDFORD PIZZA CAFE': 'BEDFORD PARK BLVD',
 'BEDLAM': 'AVENUE C',
 'BEDOUIN TENT': 'ATLANTIC AVE',
 'BEE Z CAFE': 'FRANCIS LEWIS BLVD',
 "BEECHER'S HANDMADE CHEESE": 'BROADWAY',
 'BEEFSTEWED HOUSE': 'MAIN ST',
 'BEEKMAN THEATRE': '2ND AVE',
 'BEER AUTHORITY': 'WEST   40 STREET',
 'BEER CULTURE': 'WEST 45TH STREET',
 'BEER KARMA': 'UNION AVE',
 'BEER PUB (BRIDGE PUB)': 'PENN PLZ',
 'BEER SHOP': 'AMSTERDAM AVE',
 'BEERZAAR': '6TH AVE',
 'BEETLE HOUSE': 'E 6TH ST',
 'BEETS AND CARROTS':

 'BOMBAY SANDWICH CO': 'W. 27TH STREET',
 'BOMBAY SPICE': 'BERGEN ST',
 'BOMBAY THEATRE': 'FRESH MEADOW LN',
 "BOMBAY'S": 'PEARL STREET',
 'BON APPETIT RESTAURANT': 'JAMAICA AVENUE',
 'BON SOIR CATERERS': 'EAST   63 STREET',
 'BONA PIZZA': 'WESTCHESTER AVENUE',
 'BONAFIDE CAFE & DELI': 'KANE ST',
 'BONAO BAR AND GRILL': 'JAMAICA AVENUE',
 'BONAO CHIMI RESTAURANT': 'JAMAICA AVENUE',
 'BONBON BAKERY': '69TH AVE',
 'BONCHON': 'WILLOUGHBY ST',
 'BOND 45 ITALIAN KITCHEN AND BAR': 'W 46TH ST',
 'BOND ST': 'BOND STREET',
 'BONDI SUSHI': 'W 28TH ST',
 'BONDURANTS': 'E 85TH ST',
 'BONELLE PASTRY SHOP': 'ASCAN AVENUE',
 'BONGO': 'WEST STREET',
 'BONJOUR CREPES & WINE': 'LEXINGTON AVE',
 'BONJUK': 'NORTHERN BOULEVARD',
 'BONMI VIETNAMESE': 'E 23RD ST',
 'BONNIE VEE': 'STANTON ST',
 'BONNIES GRILL': '5TH AVE',
 'BONO TRATTORIA': 'BROADWAY',
 "BONO'S CAFE": 'EAST   28 STREET',
 'BONSIGNOUR': 'JANE STREET',
 'BONUS ROOM': 'WYCKOFF AVE',
 'BOOBIETRAP': 'BLEECKER ST',
 'BOOMER AND CARTON': 'ATLANTIC A

...

 'THE BAR ROOM': 'E 60TH ST',
 'THE BARBECUE PIT': 'MOSHOLU AVENUE',
 'THE BARONESS BAR': 'CRESCENT STREET',
 'THE BASAC': 'NORTHERN BLVD',
 'THE BAYHOUSE': 'BAYSIDE DRIVE',
 'THE BEAN': '3RD AVE',
 'THE BEATRICE INN': 'W 12TH ST',
 'THE BEDFORD': 'BEDFORD AVENUE',
 'THE BEEKMAN': 'BEEKMAN STREET',
 'THE BEER GARDEN': 'VICTORY BOULEVARD',
 'THE BEERKEEPER': 'WOODSIDE AVE',
 'THE BELL HOUSE': '7 STREET',
 'THE BELVEDERE CLUB': 'HOWARD AVE',
 'THE BEST $1.00 PIZZA': 'WEST   23 STREET',
 'THE BEST ITALIAN PIZZA': 'EAST  149 STREET',
 'THE BEST OF NEW YORK FOOD': 'WATER ST',
 'THE BEST PIZZA': '9 AVENUE',
 'THE BETTERED BAGEL': 'SOUTH AVE',
 'THE BIG APPLE (RITZ CARLTON HOTEL EMPLOYEE CAFETERIA)': 'CENTRAL PARK SOUTH',
 'THE BIG SLICE': '5TH AVE',
 'THE BISTRO': 'W 37TH ST',
 'THE BISTRO (HBO CAFETERIA)': 'AVENUE OF THE AMERICAS',
 'THE BISTRO AT THE COURTYARD & RESIDENCE INN BY MARRIOTT': 'BROADWAY',
 'THE BITTER END': 'BLEECKER STREET',
 'THE BLACK DERBY': 'W 4TH ST',
 'THE BLACK SHEEP':

 'THE PREPPED LIFE': 'ROCKAWAY BLVD',
 'THE PRESS BOX': '2 AVENUE',
 'THE PRINCETON CLUB': 'WEST   43 STREET',
 'THE PUBLIC NEW YORK': 'CHRYSTIE ST',
 'THE PUNCH BOWL': 'BROADWAY',
 'THE PURPLE ROSE BAR': '8 AVENUE',
 'THE QUARTER': 'LAFAYETTE AVE',
 'THE QUAYS': '30 AVENUE',
 'THE QUEENS FOOD COURT': 'GUY R BREWER BLVD',
 'THE QUEENSBORO': 'NORTHERN BLVD',
 'THE RABBIT HOLE': 'BEDFORD AVENUE',
 'THE RAG TRADER/BOPEEP BAR': 'W 36TH ST',
 'THE RAGING SKILLET': 'EAST HOUSTON STREET',
 'THE RAIL LINE DINER': 'WEST   23 STREET',
 'THE RANCH RESTAURANT & BAR OF GUYANA': 'LIBERTY AVENUE',
 'THE RANDOLPH AT BROOME': 'BROOME STREET',
 'THE RED CAT': '10 AVENUE',
 'THE RED DOORS BAR AND GRILL': 'SURF AVE',
 'THE RED LION': 'BLEECKER STREET',
 'THE REGAL': 'HOPE STREET',
 'THE RESTAURANT': 'ROCKAWAY BEACH BOULEVARD',
 'THE RIBBON': 'W 72ND ST',
 'THE RICE NOODLE': 'BLEECKER ST',
 'THE RICHARDSON': 'GRAHAM AVENUE',
 'THE RICKEY COCKTAIL LOUNGE': 'W 55TH ST',
 'THE RIDGEWOOD ALE HOUSE': 'MYRTLE AV

 'TRYON PUBLIC HOUSE': 'BROADWAY',
 'TRYST LOUNGE': '13 AVENUE',
 'TS DELICIOUS BITES RESTAURANTS': 'CHURCH AVE',
 'TSION CAFE & BAKERY': 'SAINT NICHOLAS AVE',
 'TSOB-TSOBE CAFE LOUNGE BAR': 'CONEY ISLAND AVE',
 'TSQ BRASSERIE': '7 AVENUE',
 'TSQ FOOD COURT': 'W 48TH ST',
 'TSURUTONTAN UDON NOODLE BRASSERIE': 'W 48TH ST',
 'TSUSHIMA': 'E 44TH ST',
 'TU AREPA PIZZA CAFE': '67TH AVE',
 'TU CACHAPA': 'BROADWAY',
 'TU CASA': 'QUEENS BLVD',
 'TU CASA RESTAURANT': 'STEINWAY STREET',
 'TU CASA RESTAURANT AND CATERING': 'METROPOLITAN AVE',
 'TU SALUD ES RIQUEZA': '101ST AVE',
 'TU SONRISA': 'AUDUBON AVENUE',
 'TU TIPICO DOMINICANO': 'CASTLETON AVE',
 "TU-LU'S GLUTEN-FREE BAKERY": 'EAST   11 STREET',
 'TUBBY HOOK TAVERN': 'BROADWAY',
 'TUCK SHOP': 'LEXINGTON AVE',
 'TUDOR CITY MARKETPLACE': 'E 42ND ST',
 'TUDOR CITY STEAKHOUSE': 'TUDOR CITY PL',
 'TUE THAI FOOD': 'GREENWICH AVENUE',
 'TUFFET': 'GRAHAM AVENUE',
 'TUFINO PIZZERIA NAPOLETANA': 'DITMARS BOULEVARD',
 'TUK TUK': 'VERNON BLVD',
 'TULC

 'WAHEYO MODERN MEXICAN': 'QUEENS PLZ S',
 'WAHI DINER': 'BROADWAY',
 'WAHI OYSTER BAR / WAHIZZA': 'BROADWAY',
 'WAHLBURGERS': 'STILLWELL AVE',
 'WAI KANG CHINESE RESTAURANT': 'BEDFORD PARK BLVD E',
 'WAKAMATSU JAPANESE RESTAURANT': 'GRAND AVE',
 'WAKAMBA': '8 AVENUE',
 'WAKER CHICKEN': 'UNION TPKE',
 "WALDY'S WOOD FIRED PIZZA & PENNE": '6 AVENUE',
 "WALKER'S RESTAURANT": 'NORTH MOORE STREET',
 'WALLFLOWER': 'W 12TH ST',
 'WALLSE RESTARUANT': 'WEST 11 STREET',
 'WALNUT BUS STOP': 'EAST  134 STREET',
 'WALTER FOODS': 'GRAND STREET',
 'WALTER KERR THEATRE': 'WEST   48 STREET',
 'WALTER READE THEATER': 'LINCOLN CENTER PLAZA',
 "WALTER'S BAR": '8 AVENUE',
 "WALTER'S COTTAGE": 'W 43RD ST',
 "WALTER'S/KARASU": 'DEKALB AVENUE',
 'WAN CHAI SEAFOOD RESTAURANT': '40TH RD',
 "WAN'S GARDEN": 'FOREST AVE',
 'WANG BIG BOWL NOODLE': 'FARRINGTON ST',
 'WANG EASTERN OF ZHANG': 'E 163RD ST',
 'WANG GARDEN': 'WESTCHESTER AVENUE',
 'WANG JIANG RESTAURANT': 'JERSEY ST',
 'WANG VILLAGE SEAFOOD RESTAURANT': 

 'ZAIQA RESTAURANT': 'BATH AVE',
 'ZAMAAN': 'AVENUE X',
 'ZAMAAN HOOKAH BAR': 'BROADWAY',
 "ZANNY'S CAFE": 'COLUMBUS AVENUE',
 'ZARA CAFE GRILL': 'HYLAN BLVD',
 'ZARO': 'METROPOLITAN AVE',
 "ZARO'S BAKERY": 'EAST   42 STREET',
 "ZARO'S BREAD BASKET": 'PENNSYLVANIA STATION (AMTRACK)',
 "ZARO'S FAMILY BAKERY": 'PENN PLAZA',
 'ZATAR': 'CENTRAL AVE',
 'ZAUO': 'W 24TH ST',
 'ZAVO': '3RD AVE',
 'ZAWA JAPANESE': 'E 66TH ST',
 'ZAYTOON': '2 AVENUE',
 'ZAYTOONS RESTAURANT': 'VANDERBILT AVENUE',
 'ZAYTOUN MEDITERRANEAN FLAVORS': '82ND ST',
 'ZEBRA LOUNGE': '38 AVENUE',
 'ZEBU GRILL': 'EAST   92 STREET',
 'ZEE BAGELS': 'AVENUE U',
 'ZEF CAFE': 'BOWERY',
 'ZEFES MEXICAN RESTAURANT': 'MYRTLE AVENUE',
 "ZEFF'S PIZZERIA": 'S 3RD ST',
 'ZEMA DELI/GROCERY': 'MYRTLE AVENUE',
 'ZEN': '101ST AVE',
 'ZEN FUSION CUISINE': 'PARSONS BOULEVARD',
 'ZEN VEGETARIAN HOUSE': 'FLATBUSH AVE',
 'ZEN YAI': 'QUEENS BLVD',
 "ZENG'S RESTAURANT": 'FOURTH AVENUE',
 'ZENGO NYC': '3 AVENUE',
 'ZENKICHI': 'NORTH    6 STREET',


In [94]:
browser = Browser('chrome')
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "chromedriver"}
    return Browser("chrome", **executable_path)

In [105]:
def google_scrape(name,location):
    hits = []
    browser = init_browser()
    url = "https://google.com"
    browser.visit(url)
    browser.fill('q',"{} {}".format(name,location))
    button = browser.find_by_xpath("//*[@id='tsf']/div[2]/div/div[2]/div[2]/div/center/input[@name='btnK']")
    time.sleep(1)
    button.click()
    
    html = browser.html
    soup = bs(html, "html.parser")
    results = soup.select('div#resultStats')[0].text
    results = results.encode("utf-8")
    hits.append(results)
    browser.quit()
    
    return hits

In [106]:
for key,value in dba_loc.iteritems():
    google_scrape("'{}'".format(key),value)

ElementNotVisibleException: Message: u'element not interactable\n  (Session info: chrome=72.0.3626.109)\n  (Driver info: chromedriver=72.0.3626.69 (3c16f8a135abc0d4da2dff33804db79b849a7c38),platform=Windows NT 10.0.17134 x86_64)' 

In [ ]:
new = new_keys[0].split("results",1)[0]
newer = new.split("About",1)[1]
int(filter(str.isdigit, newer))